In [1]:
from utils import seed_all, get_device
from models import BARTTrain, EnsembleTokens, Question_Model, GetEncodings, SearchSimilar, get_latest_checkpoint

device is cpu


In [2]:
# BERT imports

device = get_device()
print(device)

from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
import os


Currently Running on cpu
cpu


In [3]:
# Inference
import glob
import torch

MODEL_STORE = os.path.join(os.getcwd(), 'models_store')

bart_model = BARTTrain().to(device)


bart_model = get_latest_checkpoint('checkpoint', bart_model, MODEL_STORE)

Loading model from checkpoint checkpoint-3100
model successfully loaded checkpoint-3100


In [4]:
# location = 
import pandas as pd
import os
df_context = pd.read_csv('data/latest_data/groups_master.csv') 
df_merged = pd.read_csv('data/latest_data/qna_gid.csv')
model_op = torch.load('data/latest_data/encoded_context_348.pt')


In [5]:
def inference(question, bart_tokenizer, bart_model, MODEL_STORE):

    # Get Pretrained BERT encodings
    ge = GetEncodings(MODEL_STORE = MODEL_STORE, type='questions')
    encoded_question = ge.encode(question, max_length=30)

    # Find top matching documents
    ss = SearchSimilar(iterator = df_context['context'].values.tolist(), filename='index.bin', embeddings=model_op, shape=768, device=device)
    similar_contexts = ss.get_n_similar_vectors(encoded_question, 3)
    similar_contexts.insert(0, question)

    combined_tokens = '</s></s>'.join(similar_contexts)

    print(f'Top similar document outputs is {combined_tokens}')

    # Prepare data for BART Inferencing

    source_encoding = tokenizer(
            combined_tokens,
            max_length=1024,
            padding='max_length',
            add_special_tokens=True,
            truncation=True,
            return_tensors="pt")
   

    # Inference BART Model
    output = bart_model(
            source_encoding['input_ids'].to(device),
            mode = 'eval')
    output = tokenizer.decode(output[0])
    print(output)
    return output


In [6]:
MODEL_STORE = os.path.join(os.getcwd(), 'models_store')
tokens = inference('What does torch.cosine loss do in pytorch?', tokenizer, bart_model, MODEL_STORE)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading model from checkpoint qna_checkpoint-4000


In [ ]:
print(tokens)

Error: Kernel is dead

In [ ]:

def get_latest_checkpoint(checkpoint, model, MODEL_STORE, q_string=''):

    checkpoints = sorted(glob.glob(f'{MODEL_STORE}/{checkpoint}*-[0-9]*'))
    if len(checkpoints):
        global_step = int(checkpoints[0].split('-')[-1])
        ckpt_name = '{}-{}'.format(checkpoint, global_step)
        print("Loading model from checkpoint %s" % ckpt_name)
        
        PATH = f'{MODEL_STORE}/{ckpt_name}/{checkpoint}{q_string}-{str(global_step)}.pt'
        # PATH = f'{MODEL_STORE}/{ckpt_name}/q_checkpoint-{str(global_step-1)}.pt'
        model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))
        print("model successfully loaded %s" % ckpt_name)
    
    else:
        print("No checkpoints available right now")
    return model

bart_model = get_latest_checkpoint('checkpoint', bart_model, MODEL_STORE)



def load_model(checkpoint, model, MODEL_STORE, q_string=''):

    checkpoints = sorted(glob.glob(f'{MODEL_STORE}/{checkpoint}*-[0-9]*'))
    global_step = int(checkpoints[0].split('-')[-1])
    ckpt_name = '{}-{}'.format(checkpoint, global_step)
    print("Loading model from checkpoint %s" % ckpt_name)
    
    PATH = f'{MODEL_STORE}/{ckpt_name}/{checkpoint}{q_string}-{str(global_step-1)}.pt'
    # PATH = f'{MODEL_STORE}/{ckpt_name}/q_checkpoint-{str(global_step-1)}.pt'
    model.load_state_dict(torch.load(PATH))
    print("model successfully loaded %s" % ckpt_name)

    return model

In [ ]:

model = EnsembleTokens().to(device)
model = get_latest_checkpoint(checkpoint='qna_checkpoint', model=model, MODEL_STORE=MODEL_STORE, q_string='')

In [ ]:
ques_model = list(model.children())[0]

In [ ]:
PATH = os.path.join(os.getcwd(), 'models_store/qna_checkpoint-4000/qna_checkpoint-4000-q.pt')

torch.save(ques_model.state_dict(),PATH)